_**Import PKGs**_

In [ ]:
import gmso 
import random
import numpy as np
import mbuild as mb
from gmso.core.forcefield import ForceField
from gmso.external import from_mbuild
from gmso.parameterization import apply
from mbuild.lib.recipes.polymer import Polymer
from rdkit.Chem import Draw
from IPython.display import display

_**Build the Structure of Repeat Unit**_

In [ ]:
comp = mb.load('[H]Cc1ccc(COC(=O)Cc2ccc(C(=O)O[H])cc2)cc1', smiles=True) 
comp.visualize()


In [ ]:
comp.save('comp.pdb')

_**Apply the Force Field to the Repeat Unit Mol**_

In [ ]:
gmso_oplsaa = ForceField("oplsaa.xml")
gmso_topology = from_mbuild(comp)
apply(top=gmso_topology, forcefields=gmso_oplsaa, identify_connections=True) 

_**Check the missing types**_

In [ ]:
#ignore the missing types
gmso_oplsaa = ForceField("oplsaa.xml")
gmso_topology = from_mbuild(comp)
apply(top=gmso_topology, forcefields=gmso_oplsaa, identify_connections=True, ignore_params=[ "dihedrals", "impropers"], remove_untyped=False)

In [ ]:
#check dihedral types
for dihedral in gmso_topology.dihedrals:
    print(dihedral.dihedral_type)

In [ ]:
for dihedral in gmso_topology.dihedrals:
    if dihedral.dihedral_type is None:
        atom_types = [a.atom_type.name for a in dihedral.connection_members]
        atom_classes = [a.atom_type.atomclass for a in dihedral.connection_members]
        print(atom_types)
        print(atom_classes)
        print()

In [ ]:
#check improper types
for improper in gmso_topology.impropers:
    print(improper.improper_type)

In [ ]:
for improper in gmso_topology.impropers:
    if improper.improper_type is None:
        atom_types = [a.atom_type.name for a in improper.connection_members]
        atom_classes = [a.atom_type.atomclass for a in improper.connection_members]
        print(atom_types)
        print(atom_classes)
        print()

_**Check the Order of Atoms**_

In [ ]:
comp_rdkmol=comp.to_rdkit()
img=Draw.MolToImage(comp_rdkmol)
display(img)

_**Build the Structure of Single chain**_

In [ ]:
chain = Polymer()

chain.add_monomer(compound=comp,
                  indices=[26, 45],
                  separation=.154,
                  replace=True)
chain.add_end_groups(mb.load('[H]Cc1ccc(COC(=O)Cc2ccc(C(=O)O[H])cc2)cc1',smiles=True), 
                     index=26,
                     separation=0.154, label="head", duplicate=False)

chain.add_end_groups(mb.load('[H]Cc1ccc(COC(=O)Cc2ccc(C(=O)O[H])cc2)cc1',smiles=True), 
                     index=45,
                     separation=0.154, label="tail", duplicate=False)
chain.build(n=20, sequence='A')
chain.visualize(show_ports=True)


In [ ]:
chain.save('chain.pdb')

_**Apply the Force Field to the Single Chain**_

In [ ]:
gmso_oplsaa = ForceField("oplsaa.xml")
gmso_topology = from_mbuild(chain)
apply(top=gmso_topology, forcefields=gmso_oplsaa, identify_connections=True) 

_**Build the Structure of System Containing Several Chains**_

In [ ]:
# the pattern we generate puts points in the xy-plane, so we'll rotate the polymer
# so that it is oriented normal to the xy-plane
chain.rotate(np.pi/2, [1, 0, 0])

# define a compound to hold all the polymers
system = mb.Compound()

# create a pattern of points to fill a disk
# patterns are generated between 0 and 1,
# and thus need to be scaled to provide appropriate spacing
pattern_disk = mb.DiskPattern(50)
pattern_disk.scale(5)

# now clone the polymer and move it to the points in the pattern
for pos in pattern_disk:
    current_polymer = mb.clone(chain)
    current_polymer.translate(pos)
    system.add(current_polymer)

system.visualize()


In [ ]:
system.save('system.pdb')

_**Apply the Force Field to the System**_

In [ ]:
gmso_oplsaa = ForceField("oplsaa.xml")
gmso_topology = from_mbuild(system)
apply(top=gmso_topology, forcefields=gmso_oplsaa, identify_connections=True) 